In [6]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [7]:
from datetime import timedelta
import datetime 
import os


from fileUtils import read_file_properties_v2,find_filesv2,list_files,get_labeled_exif
from fileUtils import get_audio,str2timestamp,query_audio
from labeling_utils import splitmp3

import pandas as pd
from pathlib import Path

from sklearn.neighbors import KDTree
import numpy as np

from pydub import AudioSegment


In [8]:
from annoy import AnnoyIndex


In [168]:
from sklearn.metrics.pairwise import paired_distances
from sklearn.metrics import pairwise_distances
from annoy import AnnoyIndex


def paired_distances_broadcast(X,Y):
    return paired_distances(np.repeat(X,Y.shape[0],axis=0), Y)


def sigmoid(X):
    return 1/(1+np.exp(-X))

def sigmoidT(X,t):
    return 1/(1+np.exp(-X*t))

def dist2sim(results,gamma=1/512):
    return np.exp(-results*gamma)

In [2]:
# output = find_filesv2(location,start_time,end_time,length,0,file_properties_df)
# sorted_filtered,start_time,end_time,start_time_org,end_time_org = output

#### find the embedding for a given image

In [6]:
# p_path="/home/enis/projects/nna/data/cameraTraps/test.txt"

# with open(p_path) as p_file:
#     photo_paths=p_file.readlines()
#     photo_paths = [i.strip() for i in photo_paths]

# other_folder=["anwr_35_2019_101RECNX_RCNX3373.JPG",
# "anwr_42_2019_100RECNX_RCNX3574.JPG",
# "ivvavik_SINP03_2019_100RECNX_IMG_3219.JPG",
# "ivvavik_SINP05_2019_100RECNX_IMG_2867.JPG",
# "ivvavik_SINP06_2019_100RECNX_IMG_1884.JPG",
# "ivvavik_SINP09_2019_100RECNX_IMG_2743.JPG",
# "prudhoe_17_2019_100RECNX_RCNX3916.JPG",]

# parent_path1="/tank/data/nna/examples/randomPhotos10k/"
# parent_path2="/tank/data/nna/examples/randomPhotos1k/"

# # photo with Caribou
# photo_paths=['anwr_37_2019_100RECNX_RCNX9317.jpg']

# given image paths finds files, code is at:
# notebooks/explore/get_audio4photos.ipynb

In [286]:
file_properties_df=pd.read_pickle("../../data/realdata_v2No_stinchcomb.pkl")

In [293]:
file_properties_df2=pd.read_pickle("../../data/stinchcomb_dataV1.pkl")

In [295]:
file_properties_df["site_id"]


,site_id,locationId,site_name,recorderId,hour_min_sec,year,month,day,region,timestamp,durationSec,timestampEnd
/tank/data/nna/real/ivvavik/AR03/2018/INP-AR-03_20180513_060000.flac,AR03,AR03,,INP-AR-03,060000,2018,05,13,ivvavik,2018-05-13 06:00:00,600,2018-05-13 06:10:00
/tank/data/nna/real/prudhoe/15/2019/S4A10283_20190531_164602.flac,15,15,,S4A10283,164602,2019,05,31,prudhoe,2019-05-31 16:46:02,4438,2019-05-31 18:00:00
/tank/data/nna/real/ivvavik/AR03/2019/INP-AR-03_20190608_000000.flac,AR03,AR03,,INP-AR-03,000000,2019,06,08,ivvavik,2019-06-08 00:00:00,600,2019-06-08 00:10:00
/tank/data/nna/real/ivvavik/SINP07/2019/SINP-07_20190622_100000.flac,SINP07,SINP07,,SINP-07,100000,2019,06,22,ivvavik,2019-06-22 10:00:00,4560,2019-06-22 11:16:00
/tank/data/nna/real/prudhoe/24/2019/S4A10301_20190526_140000.flac,24,24,,S4A10301,140000,2019,05,26,prudhoe,2019-05-26 14:00:00,4560,2019-05-26 15:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...
/tank/data/nna/real/ivvavik/AR01/2019/INP-AR-01_20190808_080000.flac,AR01,AR01,,INP-AR-01,080000,2019,08,08,ivvavik,2019-08-08 08:00:00,600,2019-08-08 08:10:00
/tank/data/nna/real/prudhoe/19/2019/S4A10258_20190621_011602.flac,19,19,,S4A10258,011602,2019,06,21,prudhoe,2019-06-21 01:16:02,4438,2019-06-21 02:30:00
/tank/data/nna/real/ivvavik/AR04/2018/INP-AR-04_20180611_140000.flac,AR04,AR04,,INP-AR-04,140000,2018,06,11,ivvavik,2018-06-11 14:00:00,600,2018-06-11 14:10:00
/tank/data/nna/real/ivvavik/AR01/2018/INP-AR-01_20180723_080000.flac,AR01,AR01,,INP-AR-01,080000,2018,07,23,ivvavik,2018-07-23 08:00:00,600,2018-07-23 08:10:00


In [312]:
oo=[]
for m in np.unique(file_properties_df2["region"].values):
    a=np.unique(file_properties_df2[file_properties_df2["region"]==m]["day"].values)
    for aa in a:
        oo.append("<option>{}</option>".format(aa))
for m in np.unique(file_properties_df["region"].values):
    a=np.unique(file_properties_df[file_properties_df["region"]==m]["day"].values)
    for aa in a:
        oo.append("<option>{}</option>".format(aa))
# np.unique(file_properties_df2["region"].values),
# a

oo.sort()
set(oo)

{'<option>01</option>',
 '<option>02</option>',
 '<option>03</option>',
 '<option>04</option>',
 '<option>05</option>',
 '<option>06</option>',
 '<option>07</option>',
 '<option>08</option>',
 '<option>09</option>',
 '<option>10</option>',
 '<option>11</option>',
 '<option>12</option>',
 '<option>13</option>',
 '<option>14</option>',
 '<option>15</option>',
 '<option>16</option>',
 '<option>17</option>',
 '<option>18</option>',
 '<option>19</option>',
 '<option>20</option>',
 '<option>21</option>',
 '<option>22</option>',
 '<option>23</option>',
 '<option>24</option>',
 '<option>25</option>',
 '<option>26</option>',
 '<option>27</option>',
 '<option>28</option>',
 '<option>29</option>',
 '<option>30</option>',
 '<option>31</option>'}

In [ ]:
for i in range(12):
    print(i)

### 1. pick a sound to use for queries

In [9]:
# sound with Caribou

!find /scratch/enis/data/nna/real/ -iname "S4A10279_20190605_091602*"

/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_FCmodel
/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_FCmodel/S4A10279_20190605_091602_FCmodel000.npy
/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg
/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_rawembeddings000.npy
/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_embeddings000.npy


In [4]:
# aircraft sound

!find /scratch/enis/data/nna/real/ -iname "S4A10255_20190507_073000*"


^C


### 1.1 find similar embeddings for given embeddings in the same location 

In [31]:
# sound with Caribou

raw_embed_name="/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_rawembeddings000.npy"
raw_embed=np.load(raw_embed_name)


In [9]:
# aircraft sound
# raw_embed_name="/scratch/enis/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000_vgg/S4A10255_20190507_073000_rawembeddings000.npy"
raw_embed_name="/scratch/enis/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000_vgg/S4A10255_20190507_073000_embeddings000.npy"

raw_embed=np.load(raw_embed_name)
raw_embed=raw_embed.astype(np.float32)


In [240]:
# S4A10255_20190507_073000-1808second
(30*60)+8


1808

### 1.2 Slice embedding for exact time of interest from a big file

In [10]:
# aircraft sound
start_seconds=(30*60)+8
raw_embed_audio=raw_embed[int(start_seconds):int(start_seconds)+60]

In [33]:

# # sound with Caribou
# start_seconds=(start_time-sorted_filtered["timestamp"])[0].total_seconds()
# raw_embed_audio=raw_embed[int(start_seconds):int(start_seconds)+60]

In [34]:
raw_embed_audio.shape

(60, 128)

In [253]:
def getFileIndex(raw_embed_name,metadata):
    aircraftIndex=0
    for em,length in metadata:
        if em==raw_embed_name:
            return aircraftIndex
        aircraftIndex=aircraftIndex+length
    return -1

NameError: name 'getFileIndex' is not defined

### 2.1 load embeddings to search from 

In [200]:
%%time
# find all files
import glob
search_path="/scratch/enis/data/nna/real/prudhoe/26/"
metadataCSV="/scratch/enis/data/nna/realMerged/"+"allEmbeddings_"+"prudhoe26_V11"+".csv"

filenamePattern="*_embeddings*.npy"

# all_embeddings=list_files("/scratch/enis/data/nna/real/anwr/37/",filenamePattern)
# aircraft sound
all_embeddings=list_files(search_path,filename=filenamePattern)
# remove original embedding from the list
# del all_embeddings[all_embeddings.index(raw_embed_name)]

embedSizes=[]
for i,embed_file in enumerate(all_embeddings):
    embed=np.load(embed_file)
    embedSizes.append(embed.shape[0])

metadata=list(zip(all_embeddings,embedSizes))
save_to_csv(metadataCSV,metadata)

386
CPU times: user 748 ms, sys: 1.82 s, total: 2.57 s
Wall time: 5min 56s


* we are trying to create a matrix, rows are queries and columns are data points in database
* values are similarity, 
* then 


### 2.1.1 this is for calculating distances ourself 

In [45]:
%%time
# load and merge embedding files
#LOAD:
embeds=[None]*len(all_embeddings)
index=0
for i,embed_file in enumerate(all_embeddings):
    embed=np.load(embed_file)
    index+=embed.shape[0]
    embeds[i]=(embed)
#MERGE:
# Faster then np.concatenate
concat_embeds=np.zeros([index,128],dtype=np.float32)
index=0
for i,embed in enumerate(embeds):
    concat_embeds[index:(index+embed.shape[0]),:]=embed[:]
    index+=embed.shape[0]

CPU times: user 996 ms, sys: 1 s, total: 2 s
Wall time: 4 s


In [117]:
concat_embeds.shape

(1736614, 128)

### 2.1.2 create an ANNOY index for files at all_path on disk

In [11]:
# vector size
f=128
  # Length of item vector that will be indexed
distance='euclidean'
#prepares annoy to build the index in the specified file instead of RAM 
#(execute before adding items, no need to save after build)
fn="/scratch/enis/data/nna/realMerged/prudhoe26_V11.ann"
fn="/scratch/enis/data/nna/realMerged/allEmbeddingsV11.ann"


In [261]:

import time
def buildOnDisk(fn,f,distance='euclidean',seed=42,treeN=16):
    t = AnnoyIndex(f, distance)
    t.on_disk_build(fn)
    index=0
    start=time.time()
    for i,embed_file in enumerate(all_embeddings):
        embed=np.load(embed_file)
        for ii in range(embed.shape[0]):
            t.add_item(index, embed[ii,:])
            index+=1
        if i%1000==0:
            end=time.time()
            print("Current index:",i,"time for 1000 index",end-start)
            start=end
        # build index
    t.set_seed(seed)
    t.build(treeN)
    return t

def loadOnDisk(fn,f,distance='euclidean'):
    t = AnnoyIndex(f, distance)
    t.load(fn)
    return t


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 19.1 µs
Compiler : 2.62 s


In [12]:

p=Path(fn)
if p.exists():
    t = loadOnDisk(fn,f,distance=distance)
else:
    t = buildOnDisk(fn,f,distance=distance,seed=42,treeN=16)

True

### 3.1 Use Annoy NNs to approximate matrix, (find highest per second, then calculate others)

In [159]:
def MatrixApproxV2(queries,t,searchDepth=2):
    #(SearchDept,QuerySelf,QueryOther)
    # first row is for first item in queries, closest point in NNs, others are aligned.
    QmatrixDistance=np.zeros((searchDepth,len(queries),len(queries)))
    QmatrixIDs=np.zeros((searchDepth,len(queries),len(queries)),dtype=np.int)
    sizeOfDataset=t.get_n_items()
    for i,q in enumerate(queries):
        IDs,Distances=t.get_nns_by_item(q, searchDepth+1,include_distances=True)
        IDs,Distances=IDs[1:],Distances[1:]
        QmatrixDistance[:,i,i]=Distances[:]
        QmatrixIDs[:,i,i]=IDs[:]
        for i2,q2 in enumerate(queries):
            if i==i2:
                continue
            for SearchDepthIndex,ID in enumerate(IDs):
                alingedID=(ID+(i2-i))
                alingedID=0 if (alingedID)<0 else alingedID
                alingedID = alingedID if alingedID<sizeOfDataset else sizeOfDataset-1
                QmatrixDistance[SearchDepthIndex,i,i2]=t.get_distance(q2,alingedID)
                QmatrixIDs[SearchDepthIndex,i,i2]=alingedID

    return QmatrixDistance,QmatrixIDs



In [254]:
aircraftFileIndex=getFileIndex(raw_embed_name,metadata)
startIndex=aircraftFileIndex+1808
queries=[i for i in range(startIndex,startIndex+10,1)]
searchDepth=2
QmatrixDistance,QmatrixIDs=MatrixApproxV2(queries,t,searchDepth=2)
averageDistance_approx=np.average(QmatrixDistance,axis=1)

In [255]:
similarities,IDs=dist2sim(averageDistance_approx).flatten(),QmatrixIDs[:,:,0].flatten()
res=list(zip(similarities,IDs))

In [256]:
len(res)

20

#### Highest scores might be consequtive, merging these as one result helps decrease amount of files to listen

In [237]:
def groupResByDistance(res,theta=5):
    # find consequtive points that are close to each other

    sortedbyInd=sorted(res,key=lambda x:x[1])
    previous=sortedbyInd[0][1]
    count=0
    series=[]
    starts=[]
    
    #last one is ignored, add a fake one
    sortedbyInd.append((sortedbyInd[-1][0],sortedbyInd[-1][1]+theta+1))
    
    for score,index in sortedbyInd:
        if index<=previous+theta:
            series.append((score,index))
        else:
            starts.append(series)
            series=[]
            series.append((score,index))
        previous=index

    return starts




total groups 19
short distances
1329648 1329953


In [257]:
ResGrouped=groupResByDistance(res,theta=5)
# print("total groups",len(ResGrouped))
# previous=0
# print("short distances")
# for score_indexes in ResGrouped:
#     if (score_indexes[-1][1]-score_indexes[0][1])>30:
#         print(score_indexes[0][1],score_indexes[-1][1])


In [258]:
ResGrouped

[[(0.3420873627434949, 199662)],
 [(0.23513666278291756, 199763),
  (0.24773782430353594, 199765),
  (0.25793467015829874, 199765)],
 [(0.22867374500591423, 199775)],
 [(0.3133193261908106, 199898), (0.26999331052534553, 199899)],
 [(0.3049009514846011, 208611)],
 [(0.21640606362211398, 274935),
  (0.24928697352579735, 274935),
  (0.22077357382274507, 274936)],
 [(0.3502257485041737, 398432)],
 [(0.2747582949889767, 403581)],
 [(0.2555197891287858, 828917)],
 [(0.3149978497165593, 879141)],
 [(0.1842298181046157, 1051635), (0.24000938942737535, 1051636)],
 [(0.2830081520104452, 1674383)],
 [(0.22750286629790317, 1686194)],
 [(0.36534653708099707, 1686206)]]

In [259]:
split_folder="/home/enis/projects/nna/data/nearestNeighbours/prudhoe26_ApproxV2/"

exp_name=""
for indexes in (ResGrouped):
    score,index=(indexes[0])
    groupLength=indexes[-1][1]-indexes[0][1]+10
    print(groupLength)
    print(score,index)

    clipAddBip([index],exp_name,split_folder,metadata,excerptLen=groupLength,reductionLen=1)

# indexes=[indexes[len(indexes)//2] for indexes,count in (starts)]
# # indexes
    


10
0.3420873627434949 199662
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000.flac 1706
12
0.23513666278291756 199763
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000.flac 1807
10
0.22867374500591423 199775
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000.flac 1819
11
0.3133193261908106 199898
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000.flac 1942
10
0.3049009514846011 208611
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_120000.flac 1657
11
0.21640606362211398 274935
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190508_221602.flac 435
10
0.3502257485041737 398432
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190511_150000.flac 2520
10
0.2747582949889767 403581
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190511_161602.flac 3109
10
0.2555197891287858 828917
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190521_003000.flac 1101
10
0.3149978497165593 879141
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190522_011602.flac 177

4562

### 3.1.1 lets calculate exact values with ANNOY to see the difference

In [ ]:
%%time
matrixList=[]
queries=[i for i in range(100,110,1)]
for i,q in enumerate(queries):
    IDs,Distances=t.get_nns_by_item(q, sizeOfDataset,include_distances=True)
    a=list(zip(IDs,Distances))
    a.sort()
    print(i)
    matrixList.append(a[:])

for a in matrixList:
    print(len(a))

matrixList2=[]
for a in matrixList:
    b=[i[1] for i in a]
    matrixList2.append(b)
bb=np.array(matrixList2)

gamma=1/450
results=np.exp(-bb*gamma)

%%time
windowSize=10
# here I used average, normally we were using np.max
windowMax=np.average(results,axis=0)
windowMean=pd.Series(windowMax).rolling(window=windowSize).mean().iloc[windowSize-1:].values

# arr = np.array([1, 3, 2, 4, 5,6,7,8,8,1,1,9])
kth=40
ind = np.argpartition(windowMean, -kth)[-kth:]
sortedbyScore=sorted(list(zip(windowMean[ind],ind)),reverse=True)

for _,index in sortedbyScore[10:]:
    
    print(np.argmax(bb[:,index:index+10],axis=0))


### 4.1.0 there are three methods for creating vectors (I was experimenting merging vectors)

In [15]:
# # Method 1   1 second 
excerptLen=1
exp_name="1second"
concat_embeds2=concat_embeds[:]
raw_embed_audio2=raw_embed_audio[:]
raw_embed_audio2=raw_embed_audio2[20:21,:]
rowN=100000

#Method 2    mean 10 second
# excerptLen=10
# exp_name="Mean10Second"
# concat_embeds2=concat_embeds.reshape(-1,10,128).mean(axis=1)
# raw_embed_audio2=raw_embed_audio.reshape(-1,10,128).mean(axis=1)
# raw_embed_audio2=raw_embed_audio2[2:3,:]
# rowN=10000

# #Method 3    concat 10 second
# excerptLen=10
# exp_name="Concat10Second"
# concat_embeds2=concat_embeds.reshape(-1,1280)
# raw_embed_audio2=raw_embed_audio.reshape(-1,1280)
# raw_embed_audio2=raw_embed_audio2[2:3,:]
# rowN=10000



#### 4.1.1 Exp: Fastest Way to calculate exact matrix, 

In [69]:
%%time
import multiprocessing

def worker(index,numberItems):
    allA=[None]*numberItems

    for i in range(numberItems):
        a=u.get_distance(i, index)
    return None

jobs = []
for index in range(10):
    numberItems=u.get_n_items()
    p = multiprocessing.Process(target=worker,args=(index,numberItems))
    jobs.append(p)
    p.start()

for p in jobs:
    p.join()

CPU times: user 52 ms, sys: 788 ms, total: 840 ms
Wall time: 2.08 s


In [119]:
%%time
# one sample/query
# res=paired_distances_broadcast(raw_embed_audio2,concat_embeds2)
# res.shape

#
# 10 queries
queryCount=10
startSecond=0
excerptLen=1

gamma=1/512
startIndex=int(startSecond/excerptLen)
endIndex=startIndex+queryCount


results = pairwise_distances(raw_embed_audio[startIndex:10,:], concat_embeds)

# results=np.exp(-results*gamma)



CPU times: user 2min 55s, sys: 17.2 s, total: 3min 12s
Wall time: 5.09 s


In [169]:
%%time
results = dist2sim(results,gamma)


CPU times: user 472 ms, sys: 2.57 s, total: 3.04 s
Wall time: 3.05 s


In [32]:
# I might have to use this method, if I want to get maximum without replacement

# %%time
# stepSize=1
# windowSize=10
# resultsMax=[]
# resultsAvg=[]

# windowMax=np.max(results2,axis=0)

# for i in range(0,results2.shape[1],stepSize):
# #     windowElements=(results2[:,i:(i+windowSize+1)])
# #     colMax=np.max(windowElements,axis=0)

#     resultAvg=np.average(colMax)
#     resultsAvg.append(resultAvg)
# #     resultsMax.append(windowMax)


CPU times: user 29.9 s, sys: 960 ms, total: 30.8 s
Wall time: 29.8 s


### 4.1.2 Run sliding window over the exact Matrix

In [491]:
%%time
windowSize=10
windowMax=np.max(results,axis=0)
windowMean=pd.Series(windowMax).rolling(window=windowSize).mean().iloc[windowSize-1:].values


CPU times: user 2.31 s, sys: 1min 3s, total: 1min 5s
Wall time: 1min 5s


In [ ]:
%%time
windowSize=10
windowMax=np.max(resultMatrix,axis=0)
windowMean=pd.Series(windowMax).rolling(window=windowSize).mean().iloc[windowSize-1:].values


#### Look into stats of results

In [203]:
from scipy import stats

stats.describe(res)

DescribeResult(nobs=1732041, minmax=(0.05875636413693428, 0.3321208477020264), mean=0.12500929506519953, variance=0.00040337682893698085, skewness=1.7658461475250447, kurtosis=8.105885715994328)

In [201]:
res=windowMean[:]

#### find kth highest scores (similarity of queries)

In [334]:
# arr = np.array([1, 3, 2, 4, 5,6,7,8,8,1,1,9])
kth=250
ind = np.argpartition(res, -kth)[-kth:]
sortedbyScore=sorted(list(zip(res[ind],ind)),reverse=True)

### generating figures for similarity scores

In [152]:
res.shape,rowN,res.shape[0]/rowN

((1732041,), 100000, 17.32041)

In [153]:
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

minY,maxY=min(res),max(res)
exp_name="slidingWindow_AverageOfMax_short20k"
name=f"Euclidian Similarity {exp_name}"
nrows=res.shape[0]//rowN
print(nrows)
fig, ax = plt.subplots(nrows=nrows,figsize=(200, 20*nrows))
for i,axes in enumerate(ax):
    axes.plot(res[i*(rowN):(i+1)*(rowN)])
    axes.set_ylim(minY,maxY)
    axes.grid(True)

ax[0].set_ylabel(f"{name}",fontsize=32)

# plt.grid(True)

# loc = plticker.MultipleLocator(base=5000) # this locator puts ticks at regular intervals
# ax.xaxis.set_major_locator(loc)

plt.tight_layout()
fig.show()
filename=name.replace(" ","_")
fig.savefig(f"/home/enis/projects/nna/results/vis/nearestNeighbour/{filename}.png")


17


In [169]:
excerptLen=10
exp_name="Concat10Second"
concat_embeds2=concat_embeds.reshape(-1,1280)
raw_embed_audio2=raw_embed_audio.reshape(-1,1280)
raw_embed_audio2=raw_embed_audio2[2:3,:]
rowN=10000

res=paired_distances_broadcast(raw_embed_audio2,concat_embeds2)
res.shape

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

name=f"Euclidian Distance {exp_name}"
nrows=res.shape[0]//rowN

fig, ax = plt.subplots(nrows=nrows,figsize=(200, 20*nrows))
for i,axes in enumerate(ax):
    axes.plot(res[i*(rowN):(i+1)*(rowN)])

ax[0].set_ylabel(f"{name}",fontsize=32)

plt.grid(True)

# loc = plticker.MultipleLocator(base=5000) # this locator puts ticks at regular intervals
# ax.xaxis.set_major_locator(loc)

plt.tight_layout()
# fig.show()
filename=name.replace(" ","_")
fig.savefig(f"/home/enis/projects/nna/results/vis/nearestNeighbour/{filename}.png")


In [232]:
len(all_embeddings),excerptLen

(385, 10)

In [310]:
# sorted([sorted(start) for start in starts])

### create corresponding clips

In [ ]:
split_folder="/home/enis/projects/nna/data/nearestNeighbours/S4A10255_20190507_073000-1808seconds/slidingWindow_AverageOfMax/"
excerptLen=1
# for indexes in (starts):
#     score,index=(indexes[len(indexes)//2])
#     print(score)
#     print(embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=1))
    
for score,index in (sortedbyScore):
#     score,index=(indexes[len(indexes)//2])
    print(score)
    print(embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=1))

# indexes=[indexes[len(indexes)//2] for indexes,count in (starts)]
# # # indexes
# clipAddBip(indexes,exp_name,split_folder,embeds,all_embeddings,excerptLen=10,reductionLen=1)
    


In [50]:
kdt = KDTree(concat_embeds2, leaf_size=30, metric='euclidean')
distances,indexes=kdt.query(raw_embed_audio2, k=10, return_distance=True)
allResults=[ (distance,index) for index,distance in (zip(indexes.flatten(),distances.flatten()))]
allResults.sort()
# distances,indexes=allResults

In [209]:
def embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=1):
    alist=[]
    countStart=0
    embed_count=index*excerptLen
    for i,em in enumerate(embeds):
        countEnd=countStart+(em.shape[0])
        if countStart<=embed_count and embed_count<=countEnd:
            startSecond=embed_count-countStart
    #             print(i,countStart,countEnd,"start second:",startSecond)
            filename=("/tank/"+"/".join(all_embeddings[i].split("/")[3:-2])+"/"+all_embeddings[i].split("/")[-2][:-4]+".flac")
#             alist.append((filename,startSecond))
            return((filename,startSecond))
#             break
        countStart=countEnd

def embedIndex2fileSecond(index,metadata,excerptLen=1):
    alist=[]
    countStart=0
    embed_count=index*excerptLen
    
    for i,meta in enumerate(metadata):
        countEnd=countStart+(meta[1])
        if countStart<=embed_count and embed_count<=countEnd:
            startSecond=embed_count-countStart
    #             print(i,countStart,countEnd,"start second:",startSecond)
            filename=("/tank/"+"/".join(meta[0].split("/")[3:-2])+"/"+meta[0].split("/")[-2][:-4]+".flac")
            return((filename,startSecond))
        countStart=countEnd

    

In [210]:
split_folder="/home/enis/projects/nna/data/nearestNeighbours/"
def clipAddBip(allIndexes,exp_name,split_folder,metadata,excerptLen,reductionLen):
    # embed_count=823804
    beep_wav="/home/enis/projects/nna/data/beep.wav"
    beep_wav = AudioSegment.from_file(beep_wav)
    beep_wav=beep_wav-30
    
    
    split_folder=(split_folder+exp_name)
    Path(split_folder).mkdir(parents=True, exist_ok=True)

    for index in allIndexes[:]:
        (filename,startSecond)=embedIndex2fileSecond(index,metadata,excerptLen=reductionLen)
        endSecond=startSecond+excerptLen
        print(filename,startSecond)
        input_file=filename
        buffer=10
        start_time="{}.{}".format((startSecond-buffer)//60,(startSecond-buffer)%60)
        end_time="{}.{}".format((endSecond+buffer)//60,(endSecond+buffer)%60)
        outputSuffix=".wav"
        output_file = splitmp3(input_file,split_folder,start_time,end_time,depth=1,backend="ffmpeg",outputSuffix=outputSuffix)

        other_wav=AudioSegment.from_file(output_file)
        start,end=(buffer*1000),((buffer+excerptLen)*1000)
        result_wav=other_wav[:start]+(beep_wav)+other_wav[start:end]+beep_wav+other_wav[end:]
        o_file=result_wav.export(output_file)




In [ ]:
#notes
!ls -alh /scratch/enis/data/nna/realMerged/allEmbeddings.ann